### 开始

In [1]:
# 检查Python版本
from sys import version_info
if version_info.major != 3:
    raise Exception('请使用Python3来完成此项目')

### 数据预处理

#### 获取数据
[Dogs vs. Cats Redux: Kernels Edition
](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

In [5]:
# download data and unzip it
from urllib.request import urlretrieve
import os
from tqdm import tqdm
from zipfile import ZipFile

train_url = "https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip"
test_url = "https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/test.zip"
sample_csv_url = "https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/sample_submission.csv"

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
        
#####

def download_dataset(url, zip_file_path, folder_path, unzip=True):
    if os.path.exists(zip_file_path):
        print("file is exist, no need download")
    else:
        print("download now")
#         urlretrieve(url, zip_file_path)
        with DLProgress(unit='B', unit_scale=True, miniters=1, desc=zip_file_path) as pbar:
            urlretrieve(url, zip_file_path, pbar.hook)
    
    if unzip:
        if os.path.exists(folder_path):
            print("files found")
        else:
            print("unzip now")
            zipf = ZipFile(zip_file_path)
            zipf.extractall()
            print("unzip end")

download_dataset(train_url, 'train.zip', 'train/')
download_dataset(test_url, 'test.zip', 'test/')

file is exist, no need download
files found
file is exist, no need download
files found


#### 分离数据集，dog和cat图片分别放入train2/dogs, train2/cats

In [29]:
import os
import shutil

def split_train_set(old_dir, new_dir):
    file_list = os.listdir(old_dir)
    file_cats = filter(lambda x:x[:3] == 'cat', file_list)
    file_dogs = filter(lambda x:x[:3] == 'dog', file_list)

    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
    os.mkdir(new_dir)
    
    dogs_path = os.path.join(new_dir, 'dogs')
    cats_path = os.path.join(new_dir, 'cats')
    os.mkdir(dogs_path)
    os.mkdir(cats_path)
    
    # 此处要注意： os.symlink(src, dst)
    # dst是从它所在的目录去选择src,所以src必须是相对于dst的relative path
    for filename in file_cats:
        os.symlink('../../'+old_dir+filename, cats_path+'/'+filename)
    
    for filename in file_dogs:
        os.symlink(old_dir+filename, dogs_path+'/'+filename)
        
    print("split over")

split_train_set('train/', 'train2/')


train2/cats
split over


### 导出特征向量

In [21]:
import os

os.popen("kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -f sample_submission.csv")